In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200830.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,8/26/2020 17:31:08,The Job,Sorcerer,Visioneers,NaN,NaN,DEGENERATE GAMBLERS,THE 1970S,I LIKE SWORDS,COCAINE STRIKES BACK!,WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!,No Theme/Free-for-all
1,8/26/2020 17:33:24,Sorcerer,The Apartment,The Job,Repo Man,Visioneers,COCAINE STRIKES BACK!,THE 1970S,I LIKE SWORDS,WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!,DEGENERATE GAMBLERS,No Theme/Free-for-all
2,8/26/2020 17:34:18,Sorcerer,Repo Man,Visioneers,The Job,The Apartment,I LIKE SWORDS,COCAINE STRIKES BACK!,THE 1970S,DEGENERATE GAMBLERS,WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!,No Theme/Free-for-all
3,8/26/2020 18:36:06,Repo Man,Sorcerer,Visioneers,The Job,NaN,THE 1970S,COCAINE STRIKES BACK!,DEGENERATE GAMBLERS,I LIKE SWORDS,WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!,NaN
4,8/26/2020 19:46:04,Sorcerer,Repo Man,Visioneers,The Apartment,The Job,DEGENERATE GAMBLERS,COCAINE STRIKES BACK!,WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!,I LIKE SWORDS,THE 1970S,No Theme/Free-for-all
5,8/27/2020 9:21:03,Repo Man,Visioneers,Sorcerer,The Apartment,The Job,THE 1970S,COCAINE STRIKES BACK!,I LIKE SWORDS,No Theme/Free-for-all,NaN,NaN
6,8/27/2020 9:49:25,Repo Man,Sorcerer,The Apartment,The Job,Visioneers,THE 1970S,WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!,COCAINE STRIKES BACK!,NaN,I LIKE SWORDS,DEGENERATE GAMBLERS


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Sorcerer', 'The Apartment', 'The Job', 'Repo Man', 'Visioneers'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Sorcerer': <Candidate('Sorcerer')>,
 'The Apartment': <Candidate('The Apartment')>,
 'The Job': <Candidate('The Job')>,
 'Repo Man': <Candidate('Repo Man')>,
 'Visioneers': <Candidate('Visioneers')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(The Job, Sorcerer, Visioneers)>,
 <Ballot(Sorcerer, The Apartment, The Job, Repo Man, Visioneers)>,
 <Ballot(Sorcerer, Repo Man, Visioneers, The Job, The Apartment)>,
 <Ballot(Repo Man, Sorcerer, Visioneers, The Job)>,
 <Ballot(Sorcerer, Repo Man, Visioneers, The Apartment, The Job)>,
 <Ballot(Repo Man, Visioneers, Sorcerer, The Apartment, The Job)>,
 <Ballot(Repo Man, Sorcerer, The Apartment, The Job, Visioneers)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate        Votes  Status
-------------  -------  --------
Sorcerer             3  Hopeful
Repo Man             3  Hopeful
The Job              1  Rejected
Visioneers           0  Rejected
The Apartment        0  Rejected

FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Sorcerer             4  Elected
Repo Man             3  Rejected
The Job              0  Rejected
Visioneers           0  Rejected
The Apartment        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                     Votes  Status
------------------------------------------  -------  --------
THE 1970S                                         3  Hopeful
DEGENERATE GAMBLERS                               2  Hopeful
COCAINE STRIKES BACK!                             1  Rejected
I LIKE SWORDS                                     1  Rejected
WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!        0  Rejected
No Theme/Free-for-all                             0  Rejected

FINAL RESULT
Candidate                                     Votes  Status
------------------------------------------  -------  --------
THE 1970S                                         5  Elected
DEGENERATE GAMBLERS                               2  Rejected
COCAINE STRIKES BACK!                             0  Rejected
I LIKE SWORDS                                     0  Rejected
WHISTLEBLOWER?! I HARDLY WHISTLE KNOW 'ER!        0  Rejected
No Theme/Free-for-all                             0  Re